### this loads my packages, preps/cleans the data, outputs 4 words n-grams, a wordcloud and important phrases

In [1]:
import bs4 as bs #this will allow us to get the data from the webpage
import pandas as pd #we love pandas!
import urllib.request #this gets the link for us
import re #this will help us with regular expressions
import contractions # use this to get rid of contractions
import string
import nltk
import nltk, re, string, collections
from nltk.util import ngrams # function for making ngrams
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer()
import numpy as np
import sumy
from sumy.summarizers.lex_rank import LexRankSummarizer
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
source = urllib.request.urlopen('https://www.churchofjesuschrist.org/study/scriptures/nt/matt/15?lang=eng').read() #gives us the elements for the talk
soup = bs.BeautifulSoup(source,'html') # this is the whole webpage
#find and treat the speaker
author = soup.find("p", {'class':'author-name'})
author = str(author)
author = re.sub('<.*?>','',author)
author= re.sub('By ', '', author)
author= re.sub('\.', '', author)
author= re.sub(' ', '_', author)
author = author.lower()
#treat the title
title = soup.find("title")
title = str(title)
title = re.sub('<.*?>','',title)
title = re.sub('\xa0','_',title)
title = re.sub(r'[^\w\s]','',title)
title = title.replace(' ', '_')
title = title.replace(':', '')
title = title.lower()
#grab the talk element
talk = soup.find("div", {"class": "body-block"})
#unpack these data elements
[x.extract() for x in talk('div')]
[x.extract() for x in talk('sup')]
[x.extract() for x in talk('span')]
def remove_tags(html):
    # parse html content
    soup = talk 
    for data in soup('header'):
        data.decompose()
    for data in soup('sup class="marker"'):
        data.decompose()
    for data in soup(['style', 'script']):
        # Remove tags
        data.decompose()
    # return data by retrieving the tag content
    return ' '.join(soup.stripped_strings)
#one more cleanup of the text
talk = remove_tags(talk)
talk = re.sub('\xa0',' ',talk)
#remove possesive apostrophe
talk = ' '.join(talk.replace("’s", '').split())
#remove possesive apostrophe
talk = ' '.join(talk.replace("’", '').split())
#remove opening quote mark
talk = ' '.join(talk.replace('“', '').split())
#remove closing quote mark
talk = ' '.join(talk.replace('”', '').split())
#treat to remove contractions
talk1=[]
for word in talk.split():
    talk1.append(contractions.fix(word)) 
talk = ' '.join(talk1).lower()
#create the file for bigrams
tokenized = talk.split()
Bigrams = ngrams(tokenized, 4)
# get the frequency of each bigram in our corpus
BigramFreq = collections.Counter(Bigrams)
# what are the ten most popular ngrams in this Spanish corpus?
lists = BigramFreq.most_common(10)
num = np.array(lists,dtype=object)
reshaped = num.reshape(10,2)
df = pd.DataFrame(reshaped, columns=['phrase','times appeared'])
df['phrase'] = df['phrase'].apply(lambda x: ' '.join(x))
file = open("{}.txt".format(title), "w", encoding="utf-8")  # append mode
file.write("common phrases:" + "\n" + str(df) + "\n")
file.close()
#parse the talk
parser = PlaintextParser.from_string(talk,Tokenizer("italian"))
#summarize the talk - we choose 5 sentences here
summarizer = LexRankSummarizer()
summary = summarizer(parser.document, 5)
summary1=[]
for sentence in summary:
    summary1.append(str(sentence))  
num = np.array(summary1,dtype=object)
reshaped = num.reshape(5)
df = pd.DataFrame(reshaped, columns=[''])
file = open("{}.txt".format(title), "a", encoding = "utf-8")  # append mode
file.write("important phrases:" + "\n"  +  str(df.values) + "\n")
file.close()
# get rid of punctuation
punctuationNoPeriod = "[" + re.sub("\.","",string.punctuation) + "]"
talk = re.sub(punctuationNoPeriod, "", talk)
#get rid of possesives
talk = ' '.join(talk.replace("’s", '').split())
# get rid of punctuation
punctuationNoPeriod = "[" + re.sub("\.","",string.punctuation) + "]"
talk = re.sub(punctuationNoPeriod, "", talk)
#remove words that are 3 or less letters - unless the word is god
talk2=[]
for w in talk.split():
    #only words that are 4+ letters long
    if len(w)>3:
        #which are not stop_words
        if w not in stop_words: 
            talk2.append(w)
    if w in ['god']:
        talk2.append(w)
talk1 = ' '.join(talk2)
talk1 = re.sub(r' +',r' ', talk1).strip()
tokens = word_tokenize(talk1)
tags = nltk.pos_tag(tokens, tagset='universal')
list_of_verbs = []
for i in range(len(tags)):
    if tags[i][1] in ['ADV','DET','ADJ','NOUN','VERB','X']:
        list_of_verbs.append(tags[i][0])
    # Import the wordcloud library
    from wordcloud import WordCloud
new_verbs=[]
for i in list_of_verbs:
    new_verbs.append(lemmatizer.lemmatize(i))
# Join the different processed titles together.
long_string = ','.join(new_verbs)

# Create a WordCloud object
wordcloud = WordCloud(background_color="white", max_words=20, contour_width=3, contour_color='steelblue')

# Generate a word cloud
wordcloud.generate(long_string)

# Visualize the word cloud
wordcloud.to_file("{}.png".format(title))